In [72]:
import torch
from tqdm.notebook import tqdm
from transformers import BertTokenizer
from torch.utils.data import TensorDataset
from transformers import BertForSequenceClassification
import pandas as pd
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler
from sklearn.model_selection import train_test_split
from transformers import AdamW, get_linear_schedule_with_warmup
import numpy as np

In [73]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
device

device(type='cuda')

In [78]:
data = pd.read_csv('./data.csv')
data.head()

,ADR,text,category
0,ADR-2022-100004022857,"# Lee, Terrilynn, Marie 000029128 I=0000 29128...",Award Letter(s)
1,ADR-2022-100004025366,BeBenefits A Service of the Department of Vete...,Award Letter(s)
2,ADR-2022-100004026667,Service Verification Letter DEPARTMENT OF VETE...,Award Letter(s)
3,ADR-2022-100004027737,AB8 Benefit Verification Letter DEPARTMENT OF ...,Award Letter(s)
4,ADR-2022-100004029343,The Prudential Insurance Company of America Pr...,Award Letter(s)


In [79]:
data.isnull().sum()

ADR           0
text        200
category      0
dtype: int64

In [80]:
data.dropna(inplace=True)
data.shape

(23555, 3)

In [81]:
data['category'].value_counts()

Tax Return - Personal              11233
Retirement Account Statement(s)     6357
Purchase Agreement                  3269
Mortgage Statement                  1651
K1s                                  590
Divorce Decree                       340
Award Letter(s)                      115
Name: category, dtype: int64

In [82]:
df = data.sample(frac=0.7, random_state=10)
df.shape

(16488, 3)

In [83]:
test = data.drop(df.index).reset_index(drop=True)
test.shape

(7067, 3)

In [84]:
test['category'].value_counts()

Tax Return - Personal              3422
Retirement Account Statement(s)    1897
Purchase Agreement                  975
Mortgage Statement                  481
K1s                                 168
Divorce Decree                       94
Award Letter(s)                      30
Name: category, dtype: int64

In [85]:
test.head()

,ADR,text,category
0,ADR-2022-100004025366,BeBenefits A Service of the Department of Vete...,Award Letter(s)
1,ADR-2022-100004026667,Service Verification Letter DEPARTMENT OF VETE...,Award Letter(s)
2,ADR-2022-100004029343,The Prudential Insurance Company of America Pr...,Award Letter(s)
3,ADR-2022-100004026927,You should contact your state or local office ...,Award Letter(s)
4,ADR-2022-200004031330,RETIREE ACCOUNT STATEMENT STATEMENT EFFECTIVE ...,Award Letter(s)


In [86]:
df.reset_index(drop=True, inplace=True)
df.head()

,ADR,text,category
0,ADR-2022-200004181213,DocuSign Envelope ID: 933891EA-4FAE-448A-AB50-...,Purchase Agreement
1,ADR-2022-100004024315,250 WEST 96TH STREET S SANCTUARY SUITE 300 IND...,Retirement Account Statement(s)
2,ADR-2022-100004060782,04/25/2022 13:24 FAX 2004 ROCKET Mortgage Bill...,Mortgage Statement
3,ADR-2022-200004025232,"Page 1 of 5 Robinhood 85 Willow Rd, Menlo Park...",Retirement Account Statement(s)
4,ADR-2022-100004073191,Name(s) Shown on Return Social Security No. WA...,Tax Return - Personal


In [87]:
possible_labels = df.category.unique()

label_dict = {}
for index, possible_label in enumerate(possible_labels):
    label_dict[possible_label] = index
label_dict

{'Purchase Agreement': 0,
 'Retirement Account Statement(s)': 1,
 'Mortgage Statement': 2,
 'Tax Return - Personal': 3,
 'Award Letter(s)': 4,
 'K1s': 5,
 'Divorce Decree': 6}

In [88]:
df['label'] = df.category.replace(label_dict)
df.head()

,ADR,text,category,label
0,ADR-2022-200004181213,DocuSign Envelope ID: 933891EA-4FAE-448A-AB50-...,Purchase Agreement,0
1,ADR-2022-100004024315,250 WEST 96TH STREET S SANCTUARY SUITE 300 IND...,Retirement Account Statement(s),1
2,ADR-2022-100004060782,04/25/2022 13:24 FAX 2004 ROCKET Mortgage Bill...,Mortgage Statement,2
3,ADR-2022-200004025232,"Page 1 of 5 Robinhood 85 Willow Rd, Menlo Park...",Retirement Account Statement(s),1
4,ADR-2022-100004073191,Name(s) Shown on Return Social Security No. WA...,Tax Return - Personal,3


In [89]:
X_train, X_val, y_train, y_val = train_test_split(df.index.values, 
                                                  df.label.values, 
                                                  test_size=0.15, 
                                                  random_state=42, 
                                                  stratify=df.label.values)

In [90]:
df['data_type'] = ['not_set']*df.shape[0]
df.head()

,ADR,text,category,label,data_type
0,ADR-2022-200004181213,DocuSign Envelope ID: 933891EA-4FAE-448A-AB50-...,Purchase Agreement,0,not_set
1,ADR-2022-100004024315,250 WEST 96TH STREET S SANCTUARY SUITE 300 IND...,Retirement Account Statement(s),1,not_set
2,ADR-2022-100004060782,04/25/2022 13:24 FAX 2004 ROCKET Mortgage Bill...,Mortgage Statement,2,not_set
3,ADR-2022-200004025232,"Page 1 of 5 Robinhood 85 Willow Rd, Menlo Park...",Retirement Account Statement(s),1,not_set
4,ADR-2022-100004073191,Name(s) Shown on Return Social Security No. WA...,Tax Return - Personal,3,not_set


In [91]:
df.loc[X_train, 'data_type'] = 'train'
df.loc[X_val, 'data_type'] = 'val'

In [92]:
df.groupby(['category', 'label', 'data_type']).count()

ADR  text
category                        label data_type            
Award Letter(s)                 4     train        72    72
                                      val          13    13
Divorce Decree                  6     train       209   209
                                      val          37    37
K1s                             5     train       359   359
                                      val          63    63
Mortgage Statement              2     train       994   994
                                      val         176   176
Purchase Agreement              0     train      1950  1950
                                      val         344   344
Retirement Account Statement(s) 1     train      3791  3791
                                      val         669   669
Tax Return - Personal           3     train      6639  6639
                                      val        1172  1172

In [93]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [94]:
encoded_data_train = tokenizer.batch_encode_plus(
    df[df.data_type=='train'].text.values, 
    add_special_tokens=True, 
    return_attention_mask=True, 
    pad_to_max_length=True, 
    max_length=512, 
    truncation=True,
    return_tensors='pt'
)

encoded_data_val = tokenizer.batch_encode_plus(
    df[df.data_type=='val'].text.values, 
    add_special_tokens=True, 
    return_attention_mask=True, 
    pad_to_max_length=True, 
    max_length=512,
    truncation=True,
    return_tensors='pt'
)

/data/mmortgage/MultiPage_Classification/classification_service/lib/python3.8/site-packages/transformers/tokenization_utils_base.py:2285: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


In [95]:
input_ids_train = encoded_data_train['input_ids']
attention_masks_train = encoded_data_train['attention_mask']
labels_train = torch.tensor(df[df.data_type=='train'].label.values)

input_ids_val = encoded_data_val['input_ids']
attention_masks_val = encoded_data_val['attention_mask']
labels_val = torch.tensor(df[df.data_type=='val'].label.values)

In [96]:
dataset_train = TensorDataset(input_ids_train, attention_masks_train, labels_train)
dataset_val = TensorDataset(input_ids_val, attention_masks_val, labels_val)

In [97]:
model = BertForSequenceClassification.from_pretrained("bert-base-uncased",
                                                      num_labels=len(label_dict),
                                                      output_attentions=False,
                                                      output_hidden_states=False)
model.to(device)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

In [98]:
batch_size=16
dataloader_train = DataLoader(dataset_train, 
                              shuffle=True,
                              num_workers=0, 
                              batch_size=batch_size)

dataloader_validation = DataLoader(dataset_val, 
                                   shuffle=False,
                                   num_workers=0, 
                                   batch_size=batch_size)

In [99]:
optimizer = AdamW(model.parameters(),
                  lr=1e-5, 
                  eps=1e-8,
                  no_deprecation_warning=True)
                  
epochs = 5

scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps=0,
                                            num_training_steps=len(dataloader_train)*epochs)

In [100]:
from sklearn.metrics import f1_score

def f1_score_func(preds, labels):
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return f1_score(labels_flat, preds_flat, average='weighted')

def accuracy_per_class(preds, labels):
    label_dict_inverse = {v: k for k, v in label_dict.items()}
    
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()

    for label in np.unique(labels_flat):
        y_preds = preds_flat[labels_flat==label]
        y_true = labels_flat[labels_flat==label]
        print(f'Class: {label_dict_inverse[label]}')
        print(f'Accuracy: {len(y_preds[y_preds==label])}/{len(y_true)}\n')

In [101]:
import random
import numpy as np

seed_val = 17
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

def evaluate(dataloader_val):

    model.eval()
    
    loss_val_total = 0
    predictions, true_vals = [], []
    
    for batch in dataloader_val:
        
        batch = tuple(b.to(device) for b in batch)
        
        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                  'labels':         batch[2],
                 }

        with torch.no_grad():        
            outputs = model(**inputs)
            
        loss = outputs.loss
        logits = outputs.logits
        loss_val_total += loss.item()

        logits = logits.detach().cpu().numpy()
        label_ids = inputs['labels'].cpu().numpy()
        predictions.append(logits)
        true_vals.append(label_ids)
    
    loss_val_avg = loss_val_total/len(dataloader_val) 
    
    predictions = np.concatenate(predictions, axis=0)
    true_vals = np.concatenate(true_vals, axis=0)
            
    return loss_val_avg, predictions, true_vals

In [103]:
from tqdm.notebook import tqdm

for epoch in tqdm(range(1, epochs+1)):
    
    model.train()
    
    loss_train_total = 0
    progress_bar = tqdm(dataloader_train, desc='Epoch {:1d}'.format(epoch), leave=False, disable=False)
    for batch in progress_bar:
        model.zero_grad()
      
        batch = tuple(b.to(device) for b in batch)
        
        inputs = {'input_ids':      batch[0],
                'attention_mask': batch[1],
                'labels':         batch[2],
                }       

        outputs = model(**inputs)
        loss = outputs.loss
        loss_train_total += loss.item()
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step()
        scheduler.step()
        progress_bar.set_postfix({'training_loss': '{:.3f}'.format(loss.item()/len(batch))})
    
    torch.save(model.state_dict(), f'./model/finetuned_BERT_epoch_{epoch}.model')
    tqdm.write(f'\nEpoch {epoch}')
    loss_train_avg = loss_train_total/len(dataloader_train)
    
    tqdm.write(f'Training loss: {loss_train_avg}')
    val_loss, predictions, true_vals = evaluate(dataloader_validation)
    val_f1 = f1_score_func(predictions, true_vals)
    tqdm.write(f'Validation loss: {val_loss}')
    tqdm.write(f'F1 Score (Weighted): {val_f1}')

  0%|          | 0/5 [00:00<?, ?it/s]

Epoch 1:   0%|          | 0/876 [00:00<?, ?it/s]


Epoch 1
Training loss: 0.027577035553268577
Validation loss: 0.02549153429779014
F1 Score (Weighted): 0.9934650138688623


Epoch 2:   0%|          | 0/876 [00:00<?, ?it/s]


Epoch 2
Training loss: 0.008266502375221177
Validation loss: 0.03222398886498394
F1 Score (Weighted): 0.9935014878353923


Epoch 3:   0%|          | 0/876 [00:00<?, ?it/s]

KeyboardInterrupt: 

### Evaluation

In [104]:
test['label'] = test.category.replace(label_dict)
test.head()

,ADR,text,category,label
0,ADR-2022-100004025366,BeBenefits A Service of the Department of Vete...,Award Letter(s),4
1,ADR-2022-100004026667,Service Verification Letter DEPARTMENT OF VETE...,Award Letter(s),4
2,ADR-2022-100004029343,The Prudential Insurance Company of America Pr...,Award Letter(s),4
3,ADR-2022-100004026927,You should contact your state or local office ...,Award Letter(s),4
4,ADR-2022-200004031330,RETIREE ACCOUNT STATEMENT STATEMENT EFFECTIVE ...,Award Letter(s),4


In [105]:
encoded_data_test = tokenizer.batch_encode_plus(
    test.text.values, 
    add_special_tokens=True, 
    return_attention_mask=True, 
    padding='max_length', 
    max_length=512, 
    truncation=True,
    return_tensors='pt'
)

In [106]:
input_ids_test = encoded_data_test['input_ids']
attention_masks_test = encoded_data_test['attention_mask']
labels_test = torch.tensor(test.label.values)
dataset_test = TensorDataset(input_ids_test, attention_masks_test, labels_test)

In [107]:
batch_size=16
dataloader_test = DataLoader(dataset_test, 
                              shuffle=True,
                              num_workers=0, 
                              batch_size=batch_size)

In [110]:
model = BertForSequenceClassification.from_pretrained("bert-base-uncased",
                                                      num_labels=len(label_dict),
                                                      output_attentions=False,
                                                      output_hidden_states=False)
model.to(device)

model.load_state_dict(torch.load('./model/finetuned_BERT_epoch_1.model', map_location=torch.device('cpu')))
model.eval()
loss, predictions, true_values = evaluate(dataloader_test)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [111]:
accuracy_per_class(predictions, true_values)

Class: Purchase Agreement
Accuracy: 964/975

Class: Retirement Account Statement(s)
Accuracy: 1886/1897

Class: Mortgage Statement
Accuracy: 474/481

Class: Tax Return - Personal
Accuracy: 3415/3422

Class: Award Letter(s)
Accuracy: 29/30

Class: K1s
Accuracy: 166/168

Class: Divorce Decree
Accuracy: 89/94



In [135]:
test[test['category']=='Divorce Decree']

,ADR,text,category,label
6973,ADR-2022-100004193898,HAGE 02 MURKIS & KLIMMEL 03/10/ 2006 15:12 314...,Divorce Decree,6
6974,ADR-2022-200004196885,"Employer's Name: MILLION DOLLAR PRODUCER, INC ...",Divorce Decree,6
6975,ADR-2022-200004190301,Law Offices laf 143 E. Wallace Avenue New Cast...,Divorce Decree,6
6976,ADR-2022-200004189963,DocuSign Envelope ID: ED099FB6-B33B-42C0-8588-...,Divorce Decree,6
6977,ADR-2022-200004189963,DocuSign Envelope ID: ED099FB6-B33B-42C0-8588-...,Divorce Decree,6
...,...,...,...,...
7062,ADR-2022-100004190135,13.4. IRS Income Tax Exemption or Exclusion. F...,Divorce Decree,6
7063,ADR-2022-100004190135,"parties shall and do mutually waive , renounce...",Divorce Decree,6
7064,ADR-2022-100004190135,"EXHIBIT ""A"" Game systems and all accessories a...",Divorce Decree,6
7065,ADR-2022-200004191074,8. The Wife is not pregnant. 9. There are thre...,Divorce Decree,6


In [143]:
text = test['text'][7063]
text

'parties shall and do mutually waive , renounce , release , and forever discharge each other from any and all rights, actions, suits, claims, demands, or obligations whatsoever , in law or in equity , which either of them had , now has , or may hereafter have against the other upon or by reason of any matter , cause , or thing existing up to the effective date of this Agreement , including , without limitation , any and all claims or rights to support , maintenance or alimony of any nature which he or she may have under the laws of the state of Florida or any other state , it being the intention of the parties that after the effective date of this Agreement , there shall be between them only such rights and obligations as are specifically provided for herein . Except as specifically provided for otherwise in this Agreement , each of the parties hereto waives , renounces , releases and forever discharges any and every estate , right , title , and interest which he or she has , or may he

In [144]:
test['category'][7063]

'Divorce Decree'

In [169]:
#text = test['text'][3000]
encoded_data_text = tokenizer.encode_plus(
    text, 
    add_special_tokens=True, 
    return_attention_mask=True, 
    pad_to_max_length=True, 
    max_length=512, 
    truncation=True,
    return_tensors='pt'
)

In [170]:
input_ids_text = encoded_data_text['input_ids']
attention_masks_text = encoded_data_text['attention_mask']
labels_text = encoded_data_text['token_type_ids']

In [171]:
model = BertForSequenceClassification.from_pretrained("bert-base-uncased",
                                                      num_labels=len(label_dict),
                                                      output_attentions=False,
                                                      output_hidden_states=False)
model.load_state_dict(torch.load('./model/finetuned_BERT_epoch_1.model', map_location=torch.device('cpu')))
model.eval()
outputs = model(input_ids_text, attention_masks_text, labels_text)
outputs

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

SequenceClassifierOutput(loss=None, logits=tensor([[-0.6873, -0.8574, -0.8432, -1.1486, -0.1381, -0.8245,  5.5987]],
       grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)

In [172]:
label_dict_inverse = {v: k for k, v in label_dict.items()}
label_dict_inverse

{0: 'Purchase Agreement',
 1: 'Retirement Account Statement(s)',
 2: 'Mortgage Statement',
 3: 'Tax Return - Personal',
 4: 'Award Letter(s)',
 5: 'K1s',
 6: 'Divorce Decree'}

In [173]:
label_dict_inverse[np.argmax(outputs.logits.detach().numpy())]

'Divorce Decree'